In [1]:
import numpy as np
from glob import glob
import os
import gc
from pathlib import Path

from matplotlib import pyplot as plt
from cycler import cycler
from matplotlib.cm import ScalarMappable
from matplotlib.colors import Normalize
from matplotlib.backends.backend_pdf import PdfPages
plt.style.use('thesis')

from scipy.optimize import curve_fit
import scipy.signal as sig
from scipy import stats

plt.rcParams.update({'figure.dpi': 150})
plt.rcParams.update({'axes.prop_cycle': cycler(color=['#5994cd', '#d74164', '#4eaa76', '#d88300', '#7365cf', \
                                                      '#c85e3e', '#83a23e', '#c851b1', '#1850a1'])})

from importlib import reload
import data_processing as dp

In [5]:
data_path = '/Users/clarke/Documents/Research/Nanospheres/Data/'
# base_path = '/Volumes/LaCie/gas_collisions/pulse_calibration/sphere_20251212/' #0908/'
base_path = '/Users/clarke/Data/gas_collisions/pulse_calibration/sphere_20251212/' #sphere_20251212/'#sphere_20251129/' #sphere_20250908/'
# drive_path = '/Volumes/LaCie/'
drive_path = '/Users/clarke/Data/'
# base_path = '/Volumes/LaCie/pulse_calibration/sphere_20250406/'

if not os.path.exists(drive_path):
    print('Error: check that the external drive is plugged in!')

folders = glob(base_path + '*')

datasets = {}

for folder in folders:
    print(folder.split(base_path)[-1])
    all_items = glob(folder + '/*')
    subfolders = np.unique(['_'.join(s.split('_')[:-1]) for s in all_items])
    sub_datasets = []
    for subfolder in subfolders:
        sub_datasets.append(subfolder.split(base_path)[-1].split('/')[-1])
        print('\t' + subfolder.split(base_path)[-1].split('/')[-1])
    datasets[folder.split(base_path)[-1]] = sub_datasets

20251215_electric_calibration_5e-8mbar_0
	20251215_d_p8e_117khz_0.5vpp_lensholder1
	20251215_d_p8e_117khz_1.5vpp_lensholder1
	20251215_d_p8e_117khz_1vpp_lensholder1
	20251215_d_p8e_117khz_2vpp_lensholder1
	20251215_d_p8e_nodrive
20251215_p4e_5e-8mbar_d137khz_1
	20251215_dfg_p4e_200ns_10v
	20251215_dfg_p4e_200ns_12.5v
	20251215_dfg_p4e_200ns_15v
	20251215_dfg_p4e_200ns_17.5v
	20251215_dfg_p4e_200ns_2.5v
	20251215_dfg_p4e_200ns_20v
	20251215_dfg_p4e_200ns_5v
	20251215_dfg_p4e_200ns_7.5v
20251215_p4e_5e-8mbar_d137khz_0
	20251215_dfg_p4e_200ns_10v
	20251215_dfg_p4e_200ns_12.5v
	20251215_dfg_p4e_200ns_15v
	20251215_dfg_p4e_200ns_17.5v
	20251215_dfg_p4e_200ns_2.5v
	20251215_dfg_p4e_200ns_20v
	20251215_dfg_p4e_200ns_5v
	20251215_dfg_p4e_200ns_7.5v
20251215_p8e_5e-8mbar_d137khz_0
	20251215_dfg_p8e_200ns_10v
	20251215_dfg_p8e_200ns_12.5v
	20251215_dfg_p8e_200ns_15v
	20251215_dfg_p8e_200ns_17.5v
	20251215_dfg_p8e_200ns_2.5v
	20251215_dfg_p8e_200ns_20v
	20251215_dfg_p8e_200ns_5v
	20251215_dfg_p8e

In [6]:
t_window = 2e-3 # the window length in ms
f_cutoff_high = 1e5 # upper cutoff frequency for the bandpass filter
f_cutoff_low = 2.5e4 # lower cutoff frequency for the bandpass filter
f_cutoff = [f_cutoff_low, f_cutoff_high]

#### Calibration of volts to meters

The mechanical susceptibility of a driven, damped harmonic oscillator with a mass $m$, damping coefficient $\gamma$, and resonant angular frequency $\omega_0$ is

$$ \chi(\omega) = \frac{1}{m} \cdot \frac{1}{\omega_0^2 - \omega^2 + i\gamma\omega}. $$

If $\omega_0 \gg \gamma$, the imaginary term in the denominator can be neglected. When subjected to a driven force of (complex) amplitude $\tilde{F}$ at an angular frequency $\omega$, the $z$ response in meters, $\tilde{Z}_m$, is given by

$$ \tilde{Z}_m = \tilde{F}\cdot\chi = \frac{\tilde{F}}{m} \cdot \frac{1}{\omega_0^2 - \omega^2}. $$

$\tilde{Z}_m$ is the product of the $z$ response measured in volts, $\tilde{Z}_V$, and a calibration factor $C_z$:

$$ \tilde{Z}_V C_z = \frac{\tilde{F}}{m} \cdot \frac{1}{\omega_0^2 - \omega^2} $$

so the calibration factor is given by

$$ C_z = \frac{\tilde{F}}{m\tilde{Z}_V} \cdot \frac{1}{\omega_0^2 - \omega^2}. $$

For convenience I swap the angular frequencies $\omega$ and $\omega_0$ out for the frequencies $f$ and $f_0$:

$$ C_z = \frac{\tilde{F}}{m\tilde{Z}_V} \cdot \frac{1}{4\pi^2(f_0^2 - f^2)}. $$

In the electric calibration data, the sphere has 8 charges and is driven at 117 kHz with 4 Vpp. The electric field at the trap focus is 79 V/m for 1 volt on the electrode. This allows the force to be calculated. The response in volts is extracted from the $z$ data stream and the resonant frequency is extracted from a Voigt profile fit. Below, all quantities that go into the calculation of the calibration factor are printed out.

In [9]:
reload(dp)

dataset_ind = 1
dataset = list(datasets.keys())[dataset_ind]

plot_path = 'figures/' + base_path.split(drive_path)[-1] + dataset + '/' + datasets[dataset][0]
Path('/'.join(plot_path.split('/')[:-1])).mkdir(parents=True, exist_ok=True)
pdf = PdfPages(plot_path + '_rejected.pdf')

nf = dp.NanoFile(base_path + dataset + '/' + datasets[dataset][1] + '_0.hdf5', f_cutoff=f_cutoff, t_window=t_window, \
                 search_window=5e-6, verbose=True)
nf.calibrate_pulse_amp()
nf.compute_and_fit_psd(file_num=0, pdf=pdf)

pdf.close()

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()